# 6.1 Python Programming

In [ ]:
import duckdb
conn = duckdb.connect("../week3/database.db")

#every cursor is doing one thing at a time
cur = conn.cursor()

In [ ]:
# write any SQL statement in python
# return all rows

cur.execute("SELECT * FROM Camp_assignment LIMIT 3")
cur.fetchall()

**EXAMPLE:** Use a query to find the number of nests per each species. Create a print statement

In [ ]:
query = """
   SELECT COUNT(*) FROM Bird_nests
   WHERE Species = ?
"""
cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall():  # DuckDB workaround
    code = row[0]
    # NOT NEEDED! prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(query, [code])  # <-- added argument here
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()

cur.close()

## Pandas
Can import Pandas and use SQL statements.

In [ ]:
import pandas as pd

In [ ]:
# read in any SQL table as a pandas df
df = pd.read_sql("SELECT * FROM Site", conn)
df.head(5)

## Cursors
Cursors are an object that mediates between submitting a query and getting the desired results. If we wanted to run several queries, we'd want more cursors. Cursors return a list.

__Example:__ Return all values

In [ ]:
# recreate cursor
conn = duckdb.connect("../week3/database.db")
cur = conn.cursor()

# 6.2 Python Programming Cont.

**EXAMPLE:** Use a select statement

In [ ]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
[t[0] for t in cur.fetchall()]

__EXAMPLE:__ Get all results. Cursor is streaming mechanism, does not store results.



In [ ]:
# return all values
cur.execute("SELECT COUNT(*) FROM Bird_nests")
print(cur.fetchall())

# return single value
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()

__NOTE:__ duckdb does not support iterators

__NOTE:__ Avoid using `SELECT * FROM` since you are relying on the selection of rows. To create a more "stable"

__EXAMPLE:__ How many nests do we have for each species?

Approach 1: First get all species, then execute a count query for each species.

In [ ]:
# create a general query, where %s will be filled in with the sp code
query = " SELECT COUNT (*) FROM Bird_nests WHERE Species = '%s' "

# apply query accross all species
cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall():
    code = row[0]
    prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(prepared_query)
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()

Approach 2: Instead of using %s, use a ? in the query. Note the `cur2.execute()` information changes, and this code is slightly shorter.

In [ ]:
# create a general query using ? instead of %s
query = " SELECT COUNT (*) FROM Bird_nests WHERE Species = ? "

# apply query accross all species
cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall():
    code = row[0]
    cur2 = conn.cursor()
    cur2.execute(query, [code])
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()

cur.close()